# Demonstration of Voronoi-Based Phase Unwrapping

This notebook demonstrates the use of the `unwrap_phase_voronoi_region_growing` function from `reconlib.phase_unwrapping.voronoi_unwrap` for 2D phase unwrapping. We will:
1. Generate a synthetic 2D wrapped phase image and its ground truth.
2. Apply the Voronoi-based unwrapping algorithm.
3. Visualize the wrapped phase, unwrapped result, ground truth, and the difference map.
4. Optionally, compare with another unwrapping algorithm (`unwrap_phase_3d_goldstein`).

## 1. Setup and Imports

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add reconlib to path (if not installed)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from reconlib.phase_unwrapping.voronoi_unwrap import unwrap_phase_voronoi_region_growing
from reconlib.phase_unwrapping.goldstein_unwrap import unwrap_phase_3d_goldstein # For optional comparison
from reconlib.plotting import plot_phase_image # For visualization

## 2. Generate Synthetic Wrapped Phase Data

We'll create a simple 2D phase map composed of two Gaussian peaks, then wrap it.

In [ ]:
def create_synthetic_wrapped_phase_2d(shape=(128, 128), peak_params=None, device='cpu'):
    """Creates a synthetic 2D wrapped phase image and its ground truth."""
    if peak_params is None:
        peak_params = [
            {'amplitude': 4 * np.pi, 'center_x': shape[1]*0.35, 'center_y': shape[0]*0.35, 'sigma_x': shape[1]*0.1, 'sigma_y': shape[0]*0.1},
            {'amplitude': -3 * np.pi, 'center_x': shape[1]*0.65, 'center_y': shape[0]*0.65, 'sigma_x': shape[1]*0.15, 'sigma_y': shape[0]*0.15}
        ]
    
    Y, X = torch.meshgrid(torch.arange(shape[0], device=device, dtype=torch.float32),
                          torch.arange(shape[1], device=device, dtype=torch.float32),
                          indexing='ij')
    
    unwrapped_phase = torch.zeros(shape, device=device, dtype=torch.float32)
    for params in peak_params:
        term_y = ((Y - params['center_y'])**2) / (2 * params['sigma_y']**2)
        term_x = ((X - params['center_x'])**2) / (2 * params['sigma_x']**2)
        unwrapped_phase += params['amplitude'] * torch.exp(-(term_x + term_y))
        
    # Wrap the phase to [-pi, pi)
    wrapped_phase = (unwrapped_phase + torch.pi) % (2 * torch.pi) - torch.pi
    
    return wrapped_phase, unwrapped_phase

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_shape_2d = (100, 100) # Slightly smaller for faster processing in notebook
voxel_size_2d = (1.0, 1.0) # Physical size of voxels/pixels

wrapped_phase_2d, ground_truth_unwrapped_2d = create_synthetic_wrapped_phase_2d(shape=image_shape_2d, device=device)

# Optional: Create a simple circular mask
center_y, center_x = image_shape_2d[0] // 2, image_shape_2d[1] // 2
radius = min(image_shape_2d) * 0.45
Y, X = torch.meshgrid(torch.arange(image_shape_2d[0], device=device),
                      torch.arange(image_shape_2d[1], device=device),
                      indexing='ij')
mask_2d = (X - center_x)**2 + (Y - center_y)**2 < radius**2

print(f"Generated wrapped phase shape: {wrapped_phase_2d.shape}")
print(f"Mask shape: {mask_2d.shape}")

### Visualize Wrapped Phase and Mask

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
plot_phase_image(wrapped_phase_2d.cpu().numpy(), title="Original Wrapped Phase (2D)", ax=axes[0])
axes[1].imshow(mask_2d.cpu().numpy(), cmap='gray')
axes[1].set_title("Binary Mask (2D)")
axes[1].axis('off')
plt.tight_layout()
plt.show()

## 3. Perform Voronoi-Based Phase Unwrapping

Now, we apply `unwrap_phase_voronoi_region_growing`.

In [ ]:
# Parameters for Voronoi unwrapping
params_voronoi = {
    'mask': mask_2d,
    'voxel_size': voxel_size_2d,
    'quality_threshold': 0.05, # Lower for potentially noisy synthetic data or sparse seeds
    'min_seed_distance': 5.0,  # Physical distance in units of voxel_size
    'neighbor_connectivity': 1, # 4-connectivity for 2D
    'max_iterations_rg': -1    # Process all possible voxels within constraints
}

print("Starting Voronoi-based unwrapping...")
unwrapped_phase_voronoi = unwrap_phase_voronoi_region_growing(
    phase_data=wrapped_phase_2d,
    **params_voronoi
)
print("Voronoi-based unwrapping finished.")

### Visualize Quality Map (Approximation)

The `_vu_compute_quality_map` is internal. We'll replicate its logic here for visualization. This is for illustrative purposes; in a real application, you might not need to visualize this directly unless debugging.

In [ ]:
def _vu_wrap_to_pi_notebook(phase_diff: torch.Tensor) -> torch.Tensor:
    return (phase_diff + torch.pi) % (2 * torch.pi) - torch.pi

def visualize_quality_map(phase: torch.Tensor, voxel_size_tuple: Tuple[float,...], mask: Optional[torch.Tensor], device_str: str) -> None:
    """Replicates logic of _vu_compute_quality_map for visualization."""
    voxel_size_t = torch.tensor(list(voxel_size_tuple), dtype=torch.float32, device=device_str)
    ndim = phase.ndim
    gradients_sq = torch.zeros_like(phase, dtype=phase.dtype)
    for i in range(ndim):
        diff_fwd = _vu_wrap_to_pi_notebook(torch.roll(phase, shifts=-1, dims=i) - phase) / voxel_size_t[i]
        diff_bwd = _vu_wrap_to_pi_notebook(phase - torch.roll(phase, shifts=1, dims=i)) / voxel_size_t[i]
        gradients_sq += diff_fwd**2 + diff_bwd**2
    quality_map_approx = 1.0 / (1.0 + gradients_sq)
    min_q, max_q = quality_map_approx.min(), quality_map_approx.max()
    if max_q > min_q: quality_map_approx = (quality_map_approx - min_q) / (max_q - min_q)
    else: quality_map_approx.fill_(0.5)
    if mask is not None: quality_map_approx.masked_fill_(~mask, 0.0)
    
    plt.figure(figsize=(6,5))
    plt.imshow(quality_map_approx.cpu().numpy(), cmap='viridis')
    plt.title('Approximated Quality Map (for Voronoi Seeds)')
    plt.colorbar(label='Quality')
    plt.axis('off')
    plt.show()

visualize_quality_map(wrapped_phase_2d, voxel_size_2d, mask_2d, str(device))

## 4. Visualizations of Unwrapping Results

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot Voronoi Unwrapped Phase
im0 = axes[0].imshow(unwrapped_phase_voronoi.cpu().numpy(), cmap='viridis')
axes[0].set_title('Voronoi Unwrapped Phase')
axes[0].axis('off')
fig.colorbar(im0, ax=axes[0], label='Phase (radians)')

# Plot Ground Truth Unwrapped Phase
im1 = axes[1].imshow(ground_truth_unwrapped_2d.cpu().numpy(), cmap='viridis')
axes[1].set_title('Ground Truth Unwrapped Phase')
axes[1].axis('off')
fig.colorbar(im1, ax=axes[1], label='Phase (radians)')

# Plot Difference
difference_voronoi = unwrapped_phase_voronoi - ground_truth_unwrapped_2d
if mask_2d is not None:
    difference_voronoi.masked_fill_(~mask_2d, 0)
im2 = axes[2].imshow(difference_voronoi.cpu().numpy(), cmap='coolwarm', vmin=-0.1, vmax=0.1) # Small range for diff
axes[2].set_title('Difference (Voronoi - GT)')
axes[2].axis('off')
fig.colorbar(im2, ax=axes[2], label='Phase Error (radians)')

plt.suptitle('Voronoi-Based Unwrapping Results', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 5. (Optional) Comparison with Goldstein Unwrapping

In [ ]:
# Goldstein unwrapping is 3D. We'll make our 2D data a single-slice 3D volume.
wrapped_phase_3d_for_goldstein = wrapped_phase_2d.unsqueeze(0) # (1, H, W)

print("Starting Goldstein unwrapping...")
unwrapped_phase_goldstein_3d = unwrap_phase_3d_goldstein(wrapped_phase_3d_for_goldstein, k_filter_strength=1.0)
unwrapped_phase_goldstein_2d = unwrapped_phase_goldstein_3d.squeeze(0) # Back to (H, W)
print("Goldstein unwrapping finished.")

# Apply mask to Goldstein result for fair comparison if mask was used for Voronoi
if mask_2d is not None:
    unwrapped_phase_goldstein_2d.masked_fill_(~mask_2d, 0)

# Visualize Goldstein results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
im0 = axes[0].imshow(unwrapped_phase_goldstein_2d.cpu().numpy(), cmap='viridis')
axes[0].set_title('Goldstein Unwrapped Phase')
axes[0].axis('off')
fig.colorbar(im0, ax=axes[0], label='Phase (radians)')

im1 = axes[1].imshow(ground_truth_unwrapped_2d.cpu().numpy(), cmap='viridis')
axes[1].set_title('Ground Truth Unwrapped Phase')
axes[1].axis('off')
fig.colorbar(im1, ax=axes[1], label='Phase (radians)')

difference_goldstein = unwrapped_phase_goldstein_2d - ground_truth_unwrapped_2d
if mask_2d is not None:
    difference_goldstein.masked_fill_(~mask_2d, 0)
im2 = axes[2].imshow(difference_goldstein.cpu().numpy(), cmap='coolwarm', vmin=-np.pi, vmax=np.pi) # Wider range for diff
axes[2].set_title('Difference (Goldstein - GT)')
axes[2].axis('off')
fig.colorbar(im2, ax=axes[2], label='Phase Error (radians)')

plt.suptitle('Goldstein Unwrapping Results', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 6. Conclusion

This notebook demonstrated the `unwrap_phase_voronoi_region_growing` function on a synthetic 2D wrapped phase image. The results, including the unwrapped phase and its difference from the ground truth, were visualized. An optional comparison with a Goldstein-based unwrapper was also performed.

The Voronoi-based method's performance depends on the quality map, seed selection strategy, and the region growing logic. The current implementation uses a simplified region growing from multiple seeds and a placeholder for advanced cell merging. For more complex phase maps, the advanced merging step (currently a stub) would be crucial for optimal performance.